In [1]:
import os
import requests
from PIL import Image
from io import BytesIO

FLICKR_API_KEY = "f2a1c7b5c34389b63dd3344d1243a05e"
SEARCH_QUERY   = "technology"
PER_PAGE       = 500      # max images per Flickr page (up to 500)
TOTAL_IMAGES   = 7000     # how many you want in total
SAVE_DIR       = "Images" # flat folder where imgs will be saved

def flickr_search(query, per_page=PER_PAGE, page=1):
    url = "https://www.flickr.com/services/rest/"
    params = {
        "method":         "flickr.photos.search",
        "api_key":        FLICKR_API_KEY,
        "text":           query,
        "format":         "json",
        "nojsoncallback": 1,
        "per_page":       per_page,
        "page":           page,
        "license":        "1,2,3,4,5,6,7",
        "safe_search":    1,
        "content_type":   1,
    }
    resp = requests.get(url, params=params, timeout=10)
    resp.raise_for_status()
    return resp.json()

def download_tech_images():
    os.makedirs(SAVE_DIR, exist_ok=True)
    downloaded = 0
    page       = 1

    while downloaded < TOTAL_IMAGES:
        data   = flickr_search(SEARCH_QUERY, PER_PAGE, page)
        photos = data.get("photos", {}).get("photo", [])
        if not photos:
            print("No more photos available; ending early.")
            break

        for photo in photos:
            if downloaded >= TOTAL_IMAGES:
                break

            # build static URL for _b (large) size
            server = photo["server"]
            pid    = photo["id"]
            sec    = photo["secret"]
            img_url = f"https://live.staticflickr.com/{server}/{pid}_{sec}_b.jpg"

            try:
                r = requests.get(img_url, timeout=5)
                r.raise_for_status()
                img = Image.open(BytesIO(r.content)).convert("RGB")
                downloaded += 1
                fname = f"new_img{downloaded}.jpg"
                img.save(os.path.join(SAVE_DIR, fname))
                if downloaded % 100 == 0:
                    print(f"→ Downloaded {downloaded}/{TOTAL_IMAGES}")
            except Exception as e:
                print(f"Error #{downloaded+1} downloading {img_url}: {e}")
                # don't increment `downloaded` here—try next photo

        page += 1

    print(f"Done! {downloaded} images saved in ./{SAVE_DIR}/")

if __name__ == "__main__":
    download_tech_images()


Error #6 downloading https://live.staticflickr.com/65535/54484349722_91c4afe2ee_b.jpg: 429 Client Error: Too Many Requests for url: https://live.staticflickr.com/65535/54484349722_91c4afe2ee_b.jpg
Error #8 downloading https://live.staticflickr.com/65535/54485190526_976d711433_b.jpg: 429 Client Error: Too Many Requests for url: https://live.staticflickr.com/65535/54485190526_976d711433_b.jpg
Error #12 downloading https://live.staticflickr.com/65535/54484226872_bb8425b595_b.jpg: 429 Client Error: Too Many Requests for url: https://live.staticflickr.com/65535/54484226872_bb8425b595_b.jpg
Error #12 downloading https://live.staticflickr.com/65535/54485346988_38402a5120_b.jpg: 429 Client Error: Too Many Requests for url: https://live.staticflickr.com/65535/54485346988_38402a5120_b.jpg
Error #15 downloading https://live.staticflickr.com/65535/54484223182_50c4c1e9ba_b.jpg: 429 Client Error: Too Many Requests for url: https://live.staticflickr.com/65535/54484223182_50c4c1e9ba_b.jpg
Error #20 do